In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from pathlib import Path

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.open_package('../federalreserve.gov-consumer_finances-inheritance')
#pkg = mp.jupyter.open_source_package('federalreserve.gov-consumer_finances-inheritance')
pkg

# Survey of Consumer Finances
`federalreserve.gov-consumer_finances-inheritance-1.1.10` Last Update: 2020-08-03T20:49:08

_References to the Survey of Consumer Finances on income, network and inheritance._


## Documentation Links

* [Codebook](https://www.federalreserve.gov/econres/files/codebk2016.txt) 
* [Homepage](https://www.federalreserve.gov/econres/scfindex.htm) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References
<ul><li> <strong><a href="https://www.federalreserve.gov/econres/files/scf2016s.zip#p16i6.dta">scf_public_16</a></strong>. Survey of consumer finances, full public dataset, 2016</li><li> <strong><a href="https://www.federalreserve.gov/econres/files/scfp2016s.zip#rscfp2016.dta">scf_extract_16</a></strong>. Survey of consumer finances, public extract, 2016</li><li> <strong><a href="https://www.federalreserve.gov/econres/files/scf2016rw1s.zip#p16_rw1.dta">scf_weights_16</a></strong>. Survey of consumer finances, replicate weights, 2016</li><ul>

In [3]:
def make_category_map(df):
    """Create or update a CSV file that has the a map from 
    codes to label, for categorical variables. """


    races = pd.DataFrame({'variable':'race', 
                      'code': list(sorted(pd.concat([df.race]).unique()))})
    educations = pd.DataFrame({'variable':'education', 
                           'code': list(sorted(pd.concat([df.ed_1,df.ed_2, 
                                                          df.ed_mother_1, df.ed_mother_2,
                                                          df.ed_father_1, df.ed_father_2]).unique()))
                          }).sort_values('code').drop_duplicates()

    occupations = pd.DataFrame({'variable':'occupation', 
                           'code': list(sorted(pd.concat([df.occ_1,df.occ_2]).unique()))
                           }).sort_values('code').drop_duplicates()

    ih_type = pd.DataFrame({'variable':'ihtype', 
                            'code':list(sorted(pd.concat([df.gi_type_1, df.gi_type_2, df.gi_type_3]).unique()))
                           }).sort_values('code').drop_duplicates()
    
    t = pd.concat([races,educations, occupations, ih_type])
    t['label'] = ''
    
    p = Path('cat_map.csv')
    
    new = t.set_index(['variable','code']).to_dict(orient='index')
    
    if p.exists():
        ex = pd.read_csv(str(p), index_col=False)
        ex.columns = ['variable','code','label']
        ex = ex.set_index(['variable','code']).fillna('').to_dict(orient='index')
        
        new.update(ex)
    
    new = pd.DataFrame.from_dict(new, orient='index').fillna('').reset_index()
    
    new.columns = ['variable','code','label']
    new.to_csv(str(p), index=False)
    
    return new

def make_categories(df, column, category, map_file):
    from pandas.api.types import CategoricalDtype
    
    t = pd.read_csv(map_file).fillna('')
    
    mp = { row.code:row.label for idx, row in t[t.variable==category].iterrows() }

    df[column] = df[column].astype(CategoricalDtype(None, ordered=True)).cat.rename_categories(mp)

In [4]:
# Main variables
scf = pkg.reference('scf_public_16').dataframe()

# Extract file. The variables in this file are created by a SAS macro, which is
# the best (only?) documentation for the meaning of the variable
# https://www.federalreserve.gov/econres/files/bulletin.macro.txt

scfe = pkg.reference('scf_extract_16').dataframe()
scfe.columns = [c.lower() for c in scfe.columns]

mm_cols = [ f"MM{i}" for i in range(1,1000) ]
wt_cols = [ f"WT1B{i}" for i in range(1,1000)]

cols = {
    'case_id': 'YY1',
    'record_id': 'Y1',
    'age_1': 'X14', # Reconciled age
    'age_2': 'X19', # Reconciled age
    'hisp': 'X7004', # Do you consider yourself to be Hispanic or Latino in culture or origin?
    'race': 'X6809', # Race of respondent
    'addtional_race': 'X6810', # Respondent offered another race categot (1) or did not (5)
    # X6402 #In 2015, did (other adult) receive any income from wages or salaries?
    #'income': 'X5729', # How much was the total income you (and your family living here) received in 2015 from all sources, before taxes and other deductions were made?
    'unusual_income': 'X7650', # Is this income unusually high or low...
    'ed_1': 'X5931', # What is the highest level of school completed or the highest degree you have received?
    'ed_2': 'X6111', # What is the highest level of school completed or the highest degree you have received?
    'ed_mother_1': 'X6032', # What is the highest level of school or the highest degree mother completed?
    'ed_father_1': 'X6033', # What is the highest level of school or the highest degree father completed?
    'ed_mother_2': 'X6132', # What is the highest level of school or the highest degree mother completed?
    'ed_father_2': 'X6133', # What is the highest level of school or the highest degree father completed?  
    'occ_1': 'X7401', # What is the official title of your job?
    'occ_2': 'X7411', # What is the official title of your job?
    'gi_other_value': 'X5818', #How much altogether were any others (inheritances) you have received?
    'fin_risk': 'X7557', # Willingness to take fnancial risks, 1 to 10
    'shop_credit' : 'X7561', # Financial shopping
    'shop_credit_2' : 'X7562', # Financial Shopping
    'fin_know': 'X7556', # Financial Knowledge
    'borrow_vacation': 'X402', # Borrow for a vacation
    'plan_horizon': 'X3008', # which of the time periods listed on this page is most important to you
    'spend_exceeds':'X7510', # would you say that your (family's) spending exceeded your (family's) income,
    'spend_exceeds_excl_house':'X7508', # Spending exceeds, after purchase of house. 
    'wt0': 'X42001' # Weight
    
}
gi_cols_packed = {
    'gi_type':  'X5803 X5808 X5813'.split(), # Was that an inheritance, a trust, or something else?
    'gi_value': 'X5804 X5809 X5814'.split(), # What was its approximate value at the time it was received?
    'gi_year':  'X5805 X5810 X5815'.split(), # In what year was it received?
    'gi_from':  'X5806 X5811 X5816'.split(), # From whom was it received?
}

gi_cols = {}
for i in range(1,4):
    for k, c in gi_cols_packed.items():
        gi_cols[f"{k}_{i}"] = c[i-1]
        
all_cols = {}
all_cols.update(cols)
all_cols.update(gi_cols)

        
# Map the column names

df =  scf[all_cols.values()].rename(columns={v:k for k,v in all_cols.items()})

# make the implicate number
df.insert(2, 'implicate_id', df.record_id - df.case_id.astype('int32')*10)   
    

df['occ'] = df[['occ_1','occ_2']].replace({0:7}).min(axis=1)

In [5]:
cm = make_category_map(df);

for c in "ed_1 ed_2 ed_mother_1 ed_mother_2 ed_father_1 ed_father_2".split():
    print(f"Making categories for {c}")
    make_categories(df, c, 'education','cat_map.csv')

make_categories(df, 'race', 'race','cat_map.csv')
    
for c in "gi_type_1 gi_type_2 gi_type_3".split():
    make_categories(df, c, 'ihtype','cat_map.csv')

df['education'] = df[['ed_1','ed_2']].replace({0:7}).min(axis=1)

# %PCTL(VAR=NORMINC,PPOINTS=0 20 40 60 80 90,TAG=NINC);
# %PCTL(VAR=NORMINC,PPOINTS=0 50 90,TAG=NINC2);
# %PCTL(VAR=NETWORTH,PPOINTS=0 10 20 30 40 50 60 70 80 90 95 99,TAG=NWPCTLE);
# %PCTL(VAR=INCOME,PPOINTS=0 10 20 30 40 50 60 70 80 90 95 99,TAG=INCPCTLE);
# %PCTL(VAR=NORMINC,PPOINTS=0 10 20 30 40 50 60 70 80 90 95 99,TAG=NINCPCTLE);
    

Making categories for ed_1
Making categories for ed_2
Making categories for ed_mother_1
Making categories for ed_mother_2
Making categories for ed_father_1
Making categories for ed_father_2


In [6]:
scf_ext = scfe[['y1','networth',  'income', 'nwcat', 'nwpctlecat', 
                'norminc', 'ninccat',  'ninc2cat', 'nincpctlecat',
                'occat1', 'occat2', 'edcl', 'lifecl', 'famstruct', 'married',  'agecl', 'housecl', 'racecl','racecl4',
                'asset', 'liq','bond', 'fin','nfin', 'debt', 'indcat', 'equity','homeeq', 'revpay','bnkruplast5', 'debt2inc', 'hsaving' , 'saved'
               ]].rename(columns={'y1':'record_id'})

df = df.merge(scf_ext, on='record_id')

df['gi_sum'] = df[['gi_value_1', 'gi_value_2', 'gi_value_3']].sum(axis=1)
df['gi_sum'] = df.gi_sum.mask(df.gi_sum<0, 0) # Remove -9 and -1 values 


# Long-term planners, for plan_horizon/X3008 answered "next few years" or longer
df['lt_planner'] = df.plan_horizon.isin([3,4,5]).astype(int)

In [7]:
# This part is expensive. 
t = df[['case_id','record_id',
        'gi_type_1', 'gi_type_2', 'gi_type_3',
        'gi_value_1', 'gi_value_2', 'gi_value_3',
        'gi_year_1', 'gi_year_2', 'gi_year_3'
       ]].set_index(['case_id','record_id'])

def n_inherit_types(r):
    return r[['gi_type_1', 'gi_type_2', 'gi_type_3']].value_counts()

nit = df.apply(n_inherit_types, axis=1).fillna(0)

df['n_gift'] = nit.gift.astype(int)
df['n_inherit'] = nit.inheritance.astype(int)
df['n_trust'] = nit.trust.astype(int)
df['n_transfer'] = df['n_gift'] + df['n_inherit'] + df['n_trust']

df['any_transfer'] = (df.n_transfer > 0).astype(int)
df['any_inherit'] = (df.n_inherit > 0).astype(int)



In [8]:
df.to_csv('scf16.csv', index=False)